# Multifactor Scoring Model

In [1]:
import pandas as pd
from tqdm import tqdm
from datetime import datetime
from pyfinlab import data_api as api

Enter your ticker universe in this Excel file (../data/multifactor_inputs.xlsx).

In [2]:
tickers = list(pd.read_excel('../data/multifactor_inputs.xlsx', engine='openpyxl', sheet_name='inputs').squeeze())

Download data. 

In [3]:
data = api.current_equity_data(
    tickers, 
    info=['LONG_COMP_NAME', 'GICS_SECTOR_NAME','BEST_PX_SALES_RATIO', 'CURRENT_TRR_1YR', 'RETURN_ON_INV_CAPITAL'], 
    api_source='bloomberg'
).sort_index()
data = data.reset_index()
data = data.replace(' US Equity', '', regex=True).set_index('TICKER')
data

,LONG_COMP_NAME,GICS_SECTOR_NAME,BEST_PX_SALES_RATIO,CURRENT_TRR_1YR,RETURN_ON_INV_CAPITAL
TICKER,,,,,
A,Agilent Technologies Inc,Health Care,6.720270,30.424360,15.347929
AAL,American Airlines Group Inc,Industrials,0.256214,1.341471,-6.471751
AAP,Advance Auto Parts Inc,Consumer Discretionary,1.283908,50.714550,9.502187
AAPL,Apple Inc,Information Technology,6.963313,33.200260,46.257607
ABBV,AbbVie Inc,Health Care,3.573838,16.076730,14.736852
...,...,...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,5.320417,19.895490,60.901504
ZBH,Zimmer Biomet Holdings Inc,Health Care,3.130503,-18.170280,4.819306
ZBRA,Zebra Technologies Corp,Information Technology,5.399640,55.439000,23.670112


Standardize the data and winsorize to reduce effects from outliers. 

In [4]:
zscore = lambda x: (x - x.mean()) / x.std(ddof=1)
zscores = data.copy().groupby(['GICS_SECTOR_NAME']).transform(zscore).clip(-3, 3).fillna(0).round(2)
zscores.columns = ['VALUE', 'MOMENTUM', 'PROFITABILITY']

Compute the multifactor scoring model.

In [5]:
zscores.insert(0, 'SCORE', ((
    - zscores['VALUE']
    + zscores['MOMENTUM']
    + zscores['PROFITABILITY']
) / len(zscores.columns)).round(2))

In [6]:
zscores = pd.concat([data.GICS_SECTOR_NAME, zscores], axis=1)

In [7]:
zscores.insert(1, 'RANK', zscores[['SCORE', 'GICS_SECTOR_NAME']].groupby(['GICS_SECTOR_NAME']).rank(pct=True).round(2))

In [8]:
zscores.insert(0, 'NAME', data[['LONG_COMP_NAME']])

In [9]:
zscores = zscores.reset_index()
zscores['TICKER'] = zscores['TICKER']
zscores = zscores.set_index('TICKER')
zscores = zscores.sort_index()
zscores

,NAME,GICS_SECTOR_NAME,RANK,SCORE,VALUE,MOMENTUM,PROFITABILITY
TICKER,,,,,,,
A,Agilent Technologies Inc,Health Care,0.60,0.07,0.32,0.51,0.01
AAL,American Airlines Group Inc,Industrials,0.23,-0.46,-1.22,-0.91,-1.70
AAP,Advance Auto Parts Inc,Consumer Discretionary,0.71,0.33,-0.51,0.79,-0.32
AAPL,Apple Inc,Information Technology,0.84,0.40,-0.10,0.18,0.92
ABBV,AbbVie Inc,Health Care,0.64,0.09,-0.38,-0.08,-0.03
...,...,...,...,...,...,...,...
YUM,Yum! Brands Inc,Consumer Discretionary,0.84,0.59,1.06,-0.18,3.00
ZBH,Zimmer Biomet Holdings Inc,Health Care,0.12,-0.54,-0.48,-1.47,-0.62
ZBRA,Zebra Technologies Corp,Information Technology,0.86,0.42,-0.39,0.83,0.04


Save multifactor scoring model results as an Excel file. 

In [10]:
report_description = 'multifactor_zscores'
today = datetime.today().strftime('%m-%d-%Y')
filename = '../excel/{}_{}.xlsx'.format(report_description, today)

# Create mew Excel file.
writer = pd.ExcelWriter(filename, engine='xlsxwriter')

# Add worksheets in the order you want them here.
zscores.sort_values(by='RANK', ascending=False).to_excel(writer, sheet_name='zscores')

# Create workbook objects
workbook = writer.book
worksheet1 = writer.sheets['zscores']

# Workbook Formats
format1 = workbook.add_format({'fg_color': '#F2F2F2'})  # Background Color Left Aligned
format1.set_align('left')
format2 = workbook.add_format({'num_format': '0.00%',  # Percentage Style
                               'fg_color': '#F2F2F2'})
format3 = workbook.add_format({'num_format': '0.0000',  # Decimal Style
                               'fg_color': '#F2F2F2'})
format4 = workbook.add_format({'fg_color': '#F2F2F2'})  # Background Color
format5 = workbook.add_format({'num_format': '0.0000',  # Decimal Style with Border
                               'fg_color': '#F2F2F2'})
format5.set_left(2)

# factor_scores
# worksheet1.hide_gridlines()
worksheet1.freeze_panes(1, 1)
worksheet1.set_column('A:A', 15.86, format1)
worksheet1.set_column('B:B', 40, format1)
worksheet1.set_column('C:C', 32, format1)
worksheet1.set_column('D:D', 23, format1)
worksheet1.set_column('E:E', 23, format1)
worksheet1.set_column('F:F', 23, format1)
worksheet1.set_column('G:G', 23, format1)
worksheet1.set_column('H:H', 23, format1)
worksheet1.set_column('I:XFD', None, None, {'hidden': True})
worksheet1.autofilter('A1:I{}'.format(zscores.shape[1] + 1))
worksheet1.set_default_row(hide_unused_rows=True)
worksheet1.conditional_format('D2:D{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})
worksheet1.conditional_format('E2:E{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})
worksheet1.conditional_format('F2:F{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale', 
                                                                      'min_color': '#63BE7B',
                                                                      'mid_color': '#FFEB84',
                                                                      'max_color': '#F8696B'})
worksheet1.conditional_format('G2:G{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})
worksheet1.conditional_format('H2:H{}'.format(zscores.shape[0] + 1), {'type': '3_color_scale'})

writer.save()